In [1]:
import cv2
val_dir = 'Face Mask Dataset/Validation'

In [2]:
import glob
import os,sys
import numpy as np
import tensorflow as tf
import time
from PIL import Image
import keras

norm_type="n1to1"
quant_dir="Face Mask Dataset/Validation"
w=128
h=128
def representative_data_gen():
    files = glob.glob(quant_dir+'/**',recursive=True)
    valid_files = []
    valid_format = [".jpg", ".jpeg", ".png", ".bmp", ".ppm", ".pgm"]
    for name in files:
        ext = os.path.splitext(name)[1].lower()
        if ext not in valid_format:
            continue
        valid_files.append(os.path.join(quant_dir, name))
    if len(valid_files) == 0:
        raise Exception("No valid files in quant_input dir {}, support format{}: ".format(quant_dir, valid_format))
    print(str(len(valid_files))+"pics")
    for path in valid_files:
        img = Image.open(path)
        img = img.resize((w,h))
        img = np.array(img).astype(np.float32)
        shape = img.shape
        if len(shape) == 2:
            continue 
            shape = (1, shape[0], shape[1], 1)
        else:
            if(shape[2]!=3):
                continue
            shape = (1, shape[0], shape[1], shape[2])
        img = img.reshape(shape)
        if norm_type is not None:
            if norm_type == "0to1":
                img = img/255.0
            elif norm_type == "n1to1":
                img = (img-128)/128
            else:
                raise Exception("Unsupported norm_type: {}".format(norm_type))
#         print(img.shape)
#         print(path)
        yield [img]

In [ ]:
model = keras.models.load_model("specify model path.h5")

In [ ]:
tflite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Provide a representative dataset to ensure we quantize correctly.
tflite_converter.representative_dataset = representative_data_gen
tflite_converter.inference_input_type = tf.float32
tflite_converter.inference_output_type = tf.float32
tflite_model = tflite_converter.convert()

In [ ]:
interpreter = tf.lite.Interpreter(model_path = "specify model ouput director.tflite")
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

In [ ]:
def get_data(dir):
    images = []
    for root, dirs, files in os.walk(dir):
        for filename in files:
            path = os.path.join(root, filename)
            img = cv2.imread(path, cv2.IMREAD_COLOR)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (128, 128))
            img = img.astype(np.float32)
            img = img / 255.

            images.append(img)
    return np.array(images)
images = get_data(val_dir)

In [ ]:
interpreter.resize_tensor_input(input_details[0]['index'], (len(images), 128, 128, 3))
interpreter.resize_tensor_input(output_details[0]['index'], (len(images), 2))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

In [ ]:
interpreter.set_tensor(input_details[0]['index'], images)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)
prediction_classes = np.argmin(tflite_model_predictions, axis=1)